## 나주시 농업기상정보시스템
* 주소 : https://weather.naju.go.kr/

In [24]:
import pandas as pd
from numpy import nan
import time

In [25]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [3]:
# 2016년 2월 : 29일
# 나머지 2월 : 28일

# 1,3,5,7,8,10,12 : 31일
# 4,6,9,11 : 30일

### bs로 크롤링하기

# 날짜, 온도 빈값 처리 확인 테스트
* 2013년 1월
* 시간 없는 날짜
    * 1/4 18시

In [29]:
# webdriver로 페이지 접속
driver = webdriver.Chrome("chromedriver_94")
url = "https://weather.naju.go.kr/agri_meteo/agri_time.html"
driver.get(url)
time.sleep(1)

all_date_list = []
all_tp_list = []
for m in [1]:
    for d in range(1,32):
        # 관측기간 xpath : //*[@id="timed_date"]
        # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
        search = driver.find_element_by_xpath('//*[@id="timed_date"]')
        button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
        # 입력창 클릭 후 내용 지우고 입력하기
        search.click()
        search.clear()
        search.send_keys(f"2013{m:0>2}{d:0>2}")
        button.click()
        time.sleep(0.5)
        # bs에 담기
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')
        
        # 시간, 기온 기록
        td = soup.find_all('td')
        
        # 시간 기록
        # 날짜만 가져오기
        for i in range(24):
            if len(td[i*13].text) == 16:
                date_list.append(td[i*13].text)
                date_list = set(date_list)
                date_list = list(date_list)
        # 날짜 순서 정렬 및 24시날짜 하루 당기기
        # 각 달 마지막 날짜 처리
        date_list.sort()
        e = d+1
        n = m+1
        if f"2013-{m:0>2}-{e:0>2} 00:00" in date_list:
            a = date_list.index(f"2013-{m:0>2}-{e:0>2} 00:00")
            date_list[a] = f"2013-{m:0>2}-{d:0>2} 24:00"
        if f"2013-{n:0>2}-01 00:00" in date_list:
            a = date_list.index(f"2013-{n:0>2}-01 00:00")
            date_list[a] = f"2013-{m:0>2}-{d:0>2} 24:00"
        length = len(date_list)
        
        # 시간 샘플이 비어있는 부분 확인하기
        # 시간 샘플
        sample_date = []
        for i in range(1,25):
            tmp = f'2013-{m:0>2}-{d:0>2} {i:0>2}:00'
            sample_date.append(tmp)
            
        # 비어 있는 부분 확인
        if length < 24:
            empty_index_list = []
            for i, one in enumerate(sample_date):
                if one not in date_list:
                    empty_index_list.append(i)
            
            # 온도 기록
            # 날짜 만큼만 온도 가져오기
            tp_list = []
            for i in range(length):
                j = i * 13 + 1
                if td[j] == nan:
                    tp_list.append(nan)
                else:
                    tp_list.append(td[j].text)
            # 날짜에 맞춰 온도 순서 뒤집기
            new_tp_list = []
            for i in range(length-1,-1,-1):
                tmp = tp_list[i]
                new_tp_list.append(tmp)
            
            # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
            for i in empty_index_list:
                date_list.insert(i, sample_date[i])
                new_tp_list.insert(i, nan)
            
        if length == 24:
            # 온도 기록
            # 날짜 만큼만 온도 가져오기
            tp_list = []
            for i in range(length):
                j = i * 13 + 1
                if td[j] == nan:
                    tp_list.append(nan)
                else:
                    tp_list.append(td[j].text)
            # 날짜에 맞춰 온도 순서 뒤집기
            new_tp_list = []
            for i in range(length-1,-1,-1):
                tmp = tp_list[i]
                new_tp_list.append(tmp)
        
        # 날짜, 온도 리스트에 추가
        all_date_list.extend(date_list)
        all_tp_list.extend(new_tp_list)

In [30]:
print(len(all_date_list), len(all_tp_list))

744 744


In [32]:
all_tp_list[720:]

['1.4',
 '1.6',
 '2.5',
 '2.6',
 '3.4',
 '2.5',
 '2.3',
 '3.0',
 '4.3',
 '7.1',
 '9.4',
 '11.7',
 '13.7',
 '13.9',
 '14.0',
 '12.5',
 '11.6',
 '11.1',
 '10.8',
 '10.6',
 '10.7',
 '10.5',
 '9.6',
 '9.2']

In [6]:
# 데이터 셋 만들기
df = pd.DataFrame({"연-월-일 시간":all_date_list, "기온":all_tp_list})
df.loc[df['기온']=='-', '기온'] = nan

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   연-월-일 시간  744 non-null    object
 1   기온        730 non-null    object
dtypes: object(2)
memory usage: 11.8+ KB


## 특정 연 월 기온 크롤링 함수

In [36]:
def ym_temp_data(year, month):
    # webdriver로 페이지 접속
    driver = webdriver.Chrome("chromedriver_94")
    url = "https://weather.naju.go.kr/agri_meteo/agri_time.html"
    driver.get(url)
    driver.implicitly_wait(10)
    
    all_date_list = []
    all_tp_list = []
    
    # 2016년이 아닌 연도
    if year != 2016:
        # 31일까지 있는 달
        if month in [1,3,5,7,8,10,12]:
            m = month
            for d in range(1,32):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 30일까지 있는 달
        if month in [4,6,9,11]:
            m = month
            for d in range(1,31):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                            
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

    # 2월 ~28일
        if month in [2]:
            m = month
            for d in range(1,29):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                        
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

    # 2016년 2월 ~29일
    if year == 2016:
        # 31일까지 있는 달
        if month in [1,3,5,7,8,10,12]:
            m = month
            for d in range(1,32):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 30일까지 있는 달
        if month in [4,6,9,11]:
            m = month
            for d in range(1,31):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                            
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

    # 2월 ~29일
        if month in [2]:
            m = month
            for d in range(1,30):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                driver.implicitly_wait(10)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                        
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)
        
    return all_date_list, all_tp_list

In [9]:
# 특정 연도, 월만 기온 크롤링
# ym_temp_data(year, month)
# 2013년 1월 확인 : 24*31 = 744
y2013m1_d, y2013m1_t = ym_temp_data(2013, 1)

In [10]:
print(len(y2013m1_d), len(y2013m1_t))

744 744


In [11]:
# 2013년 12월 확인 : 24*31 = 744
y2013m12_d, y2013m12_t = ym_temp_data(2013, 12)

In [12]:
print(len(y2013m12_d), len(y2013m12_t))

744 744


In [13]:
# 2013년 4월 확인
# 30일 : 24*30 = 720
y2013m4_d, y2013m4_t = ym_temp_data(2013, 4)
print(len(y2013m4_d), len(y2013m4_t))

720 720


In [20]:
# 2016년 2월 확인
# 29일 : 24*29 = 696
y2016m2_d, y2016m2_t = ym_temp_data(2016, 2)
print(len(y2016m2_d), len(y2016m2_t))
driver.quit()

696 696


In [37]:
# 2014년 7월 확인
y2014m7_d, y2014m7_t = ym_temp_data(2014, 7)
print(len(y2014m7_d), len(y2014m7_t))
driver.quit()

744 744


In [38]:
y2014m7_t[:24]

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '30.9',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '20.7']

## 1년간 시간에 따른 기온 크롤링 함수

In [ ]:
import pandas as pd
from numpy import nan
import time

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
# 2016년 2월 : 29일
# 나머지 2월 : 28일

# 1,3,5,7,8,10,12 : 31일
# 4,6,9,11 : 30일

# 2013년 크롤링 체크

In [22]:
year = 2013

# webdriver로 페이지 접속
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome("chromedriver_94", options=options)
url = "https://weather.naju.go.kr/agri_meteo/agri_time.html"
driver.get(url)
time.sleep(1)

all_date_list = []
all_tp_list = []

# 2016년이 아닌 연도
if year != 2016:
    # 31일까지 있는 달
    for m in [1,3,5,7,8,10,12]:
        for d in range(1,32):
            # 관측기간 xpath : //*[@id="timed_date"]
            # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
            search = driver.find_element_by_xpath('//*[@id="timed_date"]')
            button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
            # 입력창 클릭 후 내용 지우고 입력하기
            search.click()
            search.clear()
            search.send_keys(f"{year}{m:0>2}{d:0>2}")
            button.click()
            time.sleep(0.5)
            # bs에 담기
            page = driver.page_source
            soup = BeautifulSoup(page, 'lxml')

            # 시간, 기온 기록
            td = soup.find_all('td')

            # 시간 기록
            # 날짜만 가져오기
            num = int(len(td)/13)
            date_list = []
            for i in range(num):
                if len(td[i*13].text) == 16:
                    date_list.append(td[i*13].text)
                    date_list = set(date_list)
                    date_list = list(date_list)
            # 날짜 순서 정렬 및 24시날짜 하루 당기기
            # 각 달 마지막 날짜 처리
            date_list.sort()
            e = d+1
            n = m+1
            if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            if f"{year}-{n:0>2}-01 00:00" in date_list:
                a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            length = len(date_list)

            # 시간 샘플이 비어있는 부분 확인하기
            # 시간 샘플
            sample_date = []
            for i in range(1,25):
                tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                sample_date.append(tmp)

            # 비어 있는 부분 확인
            if length < 24:
                empty_index_list = []
                for i, one in enumerate(sample_date):
                    if one not in date_list:
                        empty_index_list.append(i)

                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

                # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                for i in empty_index_list:
                    date_list.insert(i, sample_date[i])
                    new_tp_list.insert(i, nan)

            if length == 24:
                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

            # 날짜, 온도 리스트에 추가
            all_date_list.extend(date_list)
            all_tp_list.extend(new_tp_list)

    # 30일까지 있는 달
    for m in [4,6,9,11]:
        for d in range(1,31):
            # 관측기간 xpath : //*[@id="timed_date"]
            # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
            search = driver.find_element_by_xpath('//*[@id="timed_date"]')
            button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
            # 입력창 클릭 후 내용 지우고 입력하기
            search.click()
            search.clear()
            search.send_keys(f"{year}{m:0>2}{d:0>2}")
            button.click()
            time.sleep(0.5)
            # bs에 담기
            page = driver.page_source
            soup = BeautifulSoup(page, 'lxml')

            # 시간, 기온 기록
            td = soup.find_all('td')

            # 시간 기록
            # 날짜만 가져오기
            num = int(len(td)/13)
            date_list = []
            for i in range(num):
                if len(td[i*13].text) == 16:
                    date_list.append(td[i*13].text)
                    date_list = set(date_list)
                    date_list = list(date_list)
            # 날짜 순서 정렬 및 24시날짜 하루 당기기
            # 각 달 마지막 날짜 처리
            date_list.sort()
            e = d+1
            n = m+1
            if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            if f"{year}-{n:0>2}-01 00:00" in date_list:
                a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            length = len(date_list)

            # 시간 샘플이 비어있는 부분 확인하기
            # 시간 샘플
            sample_date = []
            for i in range(1,25):
                tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                sample_date.append(tmp)

            # 비어 있는 부분 확인
            if length < 24:
                empty_index_list = []
                for i, one in enumerate(sample_date):
                    if one not in date_list:
                        empty_index_list.append(i)

                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

                # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                for i in empty_index_list:
                    date_list.insert(i, sample_date[i])
                    new_tp_list.insert(i, nan)

            if length == 24:
                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

            # 날짜, 온도 리스트에 추가
            all_date_list.extend(date_list)
            all_tp_list.extend(new_tp_list)

    # 2월 ~28일
    for m in [2]:
        for d in range(1,29):
            # 관측기간 xpath : //*[@id="timed_date"]
            # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
            search = driver.find_element_by_xpath('//*[@id="timed_date"]')
            button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
            # 입력창 클릭 후 내용 지우고 입력하기
            search.click()
            search.clear()
            search.send_keys(f"{year}{m:0>2}{d:0>2}")
            button.click()
            time.sleep(0.5)
            # bs에 담기
            page = driver.page_source
            soup = BeautifulSoup(page, 'lxml')

            # 시간, 기온 기록
            td = soup.find_all('td')

            # 시간 기록
            # 날짜만 가져오기
            num = int(len(td)/13)
            date_list = []
            for i in range(num):
                if len(td[i*13].text) == 16:
                    date_list.append(td[i*13].text)
                    date_list = set(date_list)
                    date_list = list(date_list)
            # 날짜 순서 정렬 및 24시날짜 하루 당기기
            # 각 달 마지막 날짜 처리
            date_list.sort()
            e = d+1
            n = m+1
            if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            if f"{year}-{n:0>2}-01 00:00" in date_list:
                a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            length = len(date_list)

            # 시간 샘플이 비어있는 부분 확인하기
            # 시간 샘플
            sample_date = []
            for i in range(1,25):
                tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                sample_date.append(tmp)

            # 비어 있는 부분 확인
            if length < 24:
                empty_index_list = []
                for i, one in enumerate(sample_date):
                    if one not in date_list:
                        empty_index_list.append(i)

                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

                # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                for i in empty_index_list:
                    date_list.insert(i, sample_date[i])
                    new_tp_list.insert(i, nan)

            if length == 24:
                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

            # 날짜, 온도 리스트에 추가
            all_date_list.extend(date_list)
            all_tp_list.extend(new_tp_list)

# 2016년 2월 ~29일
if year == 2016:
    # 31일까지 있는 달
    for m in [1,3,5,7,8,10,12]:
        for d in range(1,32):
            # 관측기간 xpath : //*[@id="timed_date"]
            # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
            search = driver.find_element_by_xpath('//*[@id="timed_date"]')
            button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
            # 입력창 클릭 후 내용 지우고 입력하기
            search.click()
            search.clear()
            search.send_keys(f"{year}{m:0>2}{d:0>2}")
            button.click()
            time.sleep(0.5)
            # bs에 담기
            page = driver.page_source
            soup = BeautifulSoup(page, 'lxml')

            # 시간, 기온 기록
            td = soup.find_all('td')

            # 시간 기록
            # 날짜만 가져오기
            num = int(len(td)/13)
            date_list = []
            for i in range(num):
                if len(td[i*13].text) == 16:
                    date_list.append(td[i*13].text)
                    date_list = set(date_list)
                    date_list = list(date_list)
            # 날짜 순서 정렬 및 24시날짜 하루 당기기
            # 각 달 마지막 날짜 처리
            date_list.sort()
            e = d+1
            n = m+1
            if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            if f"{year}-{n:0>2}-01 00:00" in date_list:
                a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            length = len(date_list)

            # 시간 샘플이 비어있는 부분 확인하기
            # 시간 샘플
            sample_date = []
            for i in range(1,25):
                tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                sample_date.append(tmp)

            # 비어 있는 부분 확인
            if length < 24:
                empty_index_list = []
                for i, one in enumerate(sample_date):
                    if one not in date_list:
                        empty_index_list.append(i)

                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

                # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                for i in empty_index_list:
                    date_list.insert(i, sample_date[i])
                    new_tp_list.insert(i, nan)

            if length == 24:
                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

            # 날짜, 온도 리스트에 추가
            all_date_list.extend(date_list)
            all_tp_list.extend(new_tp_list)

    # 30일까지 있는 달
    for m in [4,6,9,11]:
        for d in range(1,31):
            # 관측기간 xpath : //*[@id="timed_date"]
            # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
            search = driver.find_element_by_xpath('//*[@id="timed_date"]')
            button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
            # 입력창 클릭 후 내용 지우고 입력하기
            search.click()
            search.clear()
            search.send_keys(f"{year}{m:0>2}{d:0>2}")
            button.click()
            time.sleep(0.5)
            # bs에 담기
            page = driver.page_source
            soup = BeautifulSoup(page, 'lxml')

            # 시간, 기온 기록
            td = soup.find_all('td')

            # 시간 기록
            # 날짜만 가져오기
            num = int(len(td)/13)
            date_list = []
            for i in range(num):
                if len(td[i*13].text) == 16:
                    date_list.append(td[i*13].text)
                    date_list = set(date_list)
                    date_list = list(date_list)
            # 날짜 순서 정렬 및 24시날짜 하루 당기기
            # 각 달 마지막 날짜 처리
            date_list.sort()
            e = d+1
            n = m+1
            if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            if f"{year}-{n:0>2}-01 00:00" in date_list:
                a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            length = len(date_list)

            # 시간 샘플이 비어있는 부분 확인하기
            # 시간 샘플
            sample_date = []
            for i in range(1,25):
                tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                sample_date.append(tmp)

            # 비어 있는 부분 확인
            if length < 24:
                empty_index_list = []
                for i, one in enumerate(sample_date):
                    if one not in date_list:
                        empty_index_list.append(i)

                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

                # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                for i in empty_index_list:
                    date_list.insert(i, sample_date[i])
                    new_tp_list.insert(i, nan)

            if length == 24:
                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

            # 날짜, 온도 리스트에 추가
            all_date_list.extend(date_list)
            all_tp_list.extend(new_tp_list)

    # 2월 ~29일
    for m in [2]:
        for d in range(1,30):
            # 관측기간 xpath : //*[@id="timed_date"]
            # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
            search = driver.find_element_by_xpath('//*[@id="timed_date"]')
            button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
            # 입력창 클릭 후 내용 지우고 입력하기
            search.click()
            search.clear()
            search.send_keys(f"{year}{m:0>2}{d:0>2}")
            button.click()
            time.sleep(0.5)
            # bs에 담기
            page = driver.page_source
            soup = BeautifulSoup(page, 'lxml')

            # 시간, 기온 기록
            td = soup.find_all('td')

            # 시간 기록
            # 날짜만 가져오기
            num = int(len(td)/13)
            date_list = []
            for i in range(num):
                if len(td[i*13].text) == 16:
                    date_list.append(td[i*13].text)
                    date_list = set(date_list)
                    date_list = list(date_list)
            # 날짜 순서 정렬 및 24시날짜 하루 당기기
            # 각 달 마지막 날짜 처리
            date_list.sort()
            e = d+1
            n = m+1
            if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            if f"{year}-{n:0>2}-01 00:00" in date_list:
                a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
            length = len(date_list)

            # 시간 샘플이 비어있는 부분 확인하기
            # 시간 샘플
            sample_date = []
            for i in range(1,25):
                tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                sample_date.append(tmp)

            # 비어 있는 부분 확인
            if length < 24:
                empty_index_list = []
                for i, one in enumerate(sample_date):
                    if one not in date_list:
                        empty_index_list.append(i)

                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

                # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                for i in empty_index_list:
                    date_list.insert(i, sample_date[i])
                    new_tp_list.insert(i, nan)

            if length == 24:
                # 온도 기록
                # 날짜 만큼만 온도 가져오기
                tp_list = []
                for i in range(length):
                    j = i * 13 + 1
                    if td[j] == nan:
                        tp_list.append(nan)
                    else:
                        tp_list.append(td[j].text)
                # 날짜에 맞춰 온도 순서 뒤집기
                new_tp_list = []
                for i in range(length-1,-1,-1):
                    tmp = tp_list[i]
                    new_tp_list.append(tmp)

            # 날짜, 온도 리스트에 추가
            all_date_list.extend(date_list)
            all_tp_list.extend(new_tp_list)

In [23]:
# 2013년 날짜, 기온 크롤링
# 1,3,5,7,8,10,12 : 7개월 * 31일 = 217 개
# 4,6,9,11 : 4개월 * 30일 = 120 개
# 2 : 28일
# 2013년 일수 : 365일 * 24시간 = 8760 개
driver.quit()
print(len(all_date_list), len(all_tp_list))

8760 8760


# 1년간 시간에 따른 기온 크롤링 함수

In [ ]:
def year_temp_data(year):
    
    # webdriver로 페이지 접속
    driver = webdriver.Chrome("chromedriver_94")
    url = "https://weather.naju.go.kr/agri_meteo/agri_time.html"
    driver.get(url)
    time.sleep(1)
    
    all_date_list = []
    all_tp_list = []
    
    # 2016년이 아닌 연도
    if year != 2016:
        # 31일까지 있는 달
        for m in [1,3,5,7,8,10,12]:
            for d in range(1,32):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                time.sleep(0.5)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 30일까지 있는 달
        for m in [4,6,9,11]:
            for d in range(1,31):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                time.sleep(0.5)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                            
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 2월 ~28일
        for m in [2]:
            for d in range(1,29):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                time.sleep(0.5)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                        
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

    # 2016년 2월 ~29일
    if year == 2016:
        # 31일까지 있는 달
        for m in [1,3,5,7,8,10,12]:
            for d in range(1,32):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                time.sleep(0.5)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 30일까지 있는 달
        for m in [4,6,9,11]:
            for d in range(1,31):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                time.sleep(0.5)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                            
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)

        # 2월 ~29일
        for m in [2]:
            for d in range(1,30):
                # 관측기간 xpath : //*[@id="timed_date"]
                # 조회 버튼 xpath : //*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img
                search = driver.find_element_by_xpath('//*[@id="timed_date"]')
                button = driver.find_element_by_xpath('//*[@id="contents"]/ul/li[2]/form/div/ul[2]/li/img')
                # 입력창 클릭 후 내용 지우고 입력하기
                search.click()
                search.clear()
                search.send_keys(f"{year}{m:0>2}{d:0>2}")
                button.click()
                time.sleep(0.5)
                # bs에 담기
                page = driver.page_source
                soup = BeautifulSoup(page, 'lxml')

                # 시간, 기온 기록
                td = soup.find_all('td')

                # 시간 기록
                # 날짜만 가져오기
                num = int(len(td)/13)
                date_list = []
                for i in range(num):
                    if len(td[i*13].text) == 16:
                        date_list.append(td[i*13].text)
                        date_list = set(date_list)
                        date_list = list(date_list)
                # 날짜 순서 정렬 및 24시날짜 하루 당기기
                # 각 달 마지막 날짜 처리
                date_list.sort()
                e = d+1
                n = m+1
                if f"{year}-{m:0>2}-{e:0>2} 00:00" in date_list:
                    a = date_list.index(f"{year}-{m:0>2}-{e:0>2} 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                if f"{year}-{n:0>2}-01 00:00" in date_list:
                    a = date_list.index(f"{year}-{n:0>2}-01 00:00")
                    date_list[a] = f"{year}-{m:0>2}-{d:0>2} 24:00"
                length = len(date_list)

                # 시간 샘플이 비어있는 부분 확인하기
                # 시간 샘플
                sample_date = []
                for i in range(1,25):
                    tmp = f'{year}-{m:0>2}-{d:0>2} {i:0>2}:00'
                    sample_date.append(tmp)

                # 비어 있는 부분 확인
                if length < 24:
                    empty_index_list = []
                    for i, one in enumerate(sample_date):
                        if one not in date_list:
                            empty_index_list.append(i)

                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)

                    # 비어 있는 날짜 삽입, 해당 날짜 온도 nan 삽입
                    for i in empty_index_list:
                        date_list.insert(i, sample_date[i])
                        new_tp_list.insert(i, nan)

                if length == 24:
                    # 온도 기록
                    # 날짜 만큼만 온도 가져오기
                    tp_list = []
                    for i in range(length):
                        j = i * 13 + 1
                        if td[j] == nan:
                            tp_list.append(nan)
                        else:
                            tp_list.append(td[j].text)
                    # 날짜에 맞춰 온도 순서 뒤집기
                    new_tp_list = []
                    for i in range(length-1,-1,-1):
                        tmp = tp_list[i]
                        new_tp_list.append(tmp)
                        
                # 날짜, 온도 리스트에 추가
                all_date_list.extend(date_list)
                all_tp_list.extend(new_tp_list)
        
    return all_date_list, all_tp_list

In [ ]:
# 2013년 날짜, 기온 크롤링
# 1,3,5,7,8,10,12 : 7개월 * 31일 = 217 개
# 4,6,9,11 : 4개월 * 30일 = 120 개
# 2 : 28일
# 2013년 일수 : 365일 * 24시간 = 8760 개
# 함수 year_temp_data(year)
date2013, temp2013 = year_temp_data(2013)
print(len(date2013), len(temp2013))